In [1]:
import cv2
import os
import sys
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import resize
import os
from os.path import join
from sklearn.model_selection import train_test_split

In [2]:
from keras.models import Sequential,Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils
from keras.regularizers import l2
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#(545, 535, 3) исходное
# file_names = sorted(os.listdir('./data/all_ica_img/'))
#IMG_SIZE = (197,197,3)
#IMG_SIZE = (224,224,3)
IMG_SIZE = (75,75,3)

In [4]:
path = './data/train_test/train/'
file_names = sorted(os.listdir(path))
X_train = [resize(cv2.imread(join(path, file_name))[25:570, 185:720], IMG_SIZE, mode='reflect')
              for file_name in file_names]
y_train = [file_name.split('.')[0].split('_')[-1] for file_name in file_names]

X_train = np.array(X_train).astype(np.float32)
y_train = np.array(y_train).astype(int)
y_train = to_categorical(y_train)

In [ ]:
path = './data/train_test/test/'
file_names = sorted(os.listdir(path))
X_test = [resize(cv2.imread(join(path, file_name))[25:570, 185:720], IMG_SIZE, mode='reflect')
              for file_name in file_names]
y_test = [file_name.split('.')[0].split('_')[-1] for file_name in file_names]

X_test = np.array(X_test).astype(np.float32)
X_test = np.array(X_test).astype(int)
y_test = to_categorical(y_test)

In [5]:
from keras.applications.xception import Xception

In [12]:
basemodel = Xception(include_top=False, input_shape=IMG_SIZE, pooling='avg')
base_out = basemodel.output
base_out = Dense(1024, activation='elu',
                       kernel_initializer='glorot_normal',
                       kernel_regularizer=l2(1e-3))(base_out)
base_out = Dropout(0.5)(base_out)

predictions = Dense(2, activation='softmax',
                          kernel_initializer='glorot_normal',
                          kernel_regularizer=l2(1e-3))(base_out)

model = Model(inputs=basemodel.input, outputs=predictions)

trainable_border = 107  # 141
for layer in model.layers[:trainable_border]:
        layer.trainable = False
for layer in model.layers[trainable_border:]:
        layer.trainable = True
        
model.compile(optimizer=Adam(lr=1e-3, decay=1e-2),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])        

In [13]:
history = model.fit(X_train[:10], y_train[:10], batch_size=10, epochs=10,
          verbose=1, validation_data=(X_train[:10], y_train[:10]))

Train on 10 samples, validate on 10 samples
Epoch 1/10
10/10 [==============================] - 4s 417ms/step - loss: 1.8558 - acc: 0.5000 - val_loss: 7.1067 - val_acc: 0.6000
Epoch 2/10
10/10 [==============================] - 0s 9ms/step - loss: 3.2275 - acc: 0.7000 - val_loss: 5.8153 - val_acc: 0.6000
Epoch 3/10
10/10 [==============================] - 0s 9ms/step - loss: 1.6049 - acc: 0.7000 - val_loss: 2.6257 - val_acc: 0.6000
Epoch 4/10
10/10 [==============================] - 0s 9ms/step - loss: 1.6013 - acc: 0.6000 - val_loss: 1.8063 - val_acc: 0.6000
Epoch 5/10
10/10 [==============================] - 0s 9ms/step - loss: 1.5470 - acc: 0.7000 - val_loss: 1.6305 - val_acc: 0.6000
Epoch 6/10
10/10 [==============================] - 0s 9ms/step - loss: 1.5125 - acc: 0.8000 - val_loss: 1.5754 - val_acc: 0.6000
Epoch 7/10
10/10 [==============================] - 0s 9ms/step - loss: 1.4841 - acc: 1.0000 - val_loss: 1.5496 - val_acc: 0.7000
Epoch 8/10
10/10 [==========================